# Lab05: Biomass Accumulation and Carbon Budget 

<div class="alert alert-warning">
If figures/tables are not rendered correctly, make sure your notebook is `trusted` by choosing `Trust Notebook` command found in `View` - `Activate Command Palette` menu. Typing in the first letters (i.e. "tr") should give you the command.
</div>

<div class="alert alert-info">
This is a student version of the notebook that some portions of code are redacted with comments like `#= .. =#`. Students are encouraged to complete the code to get same/similar results as rendered in the notebook. 
</div>

In [ ]:
using Cropbox

## 5.1 Learning Objectives

* Begin to explore ways to break down a plant growth model into pieces based on underlying physiological processes
* Incorporate physiology into the simple growth models to be more process-based.
* Evaluate model behavior and the relationships between processes determining plant biomass dynamics
* Refer to Table 5.1 in the handout for variables, parameters, and their values to use

Previously, you built a model to simulate the growth of a whole-plant using the relative growth rate (RGR) along that showed an exponential growth pattern and later in combination with a known final size using the logistic equation. Here, you will modify and expand the model to represent some basic physiological mechanisms. In the logistic growth model, we assumed that the growth rate slows down as the plant reaches its potential biomass (this is analogous to running out of space or resources in a density dependent population). There could be other reasons for plants to slow down their growth rate. One possible mechanism is that the efficiency of growth may decline over time as the plant ages. For example, the efficiency of resource acquisition and use (e.g. light) per carbon invested in foliage tissue will slow down as plants grow bigger with increasingly overlapping foliage that results in self-shading. In addition, the efficacy of a leaf, even if it's not shaded, for producing photosynthates would also decline over time as the leaf ages. At the lower level of organizations, a constant turn over of tissues, cells, proteins and metabolites takes place. That is, old tissue, cells, or molecules die or degrade while new tissue, cells, or molecules are synthesized. Here we will explore some of the simple but physiologically meaningful ways to model realistic plant growth patterns. As done in the previous examples, we will be simulating biomass accumulation of a whole-plant from seed to maturity for an annual crop.

## 5.2 Growth efficiency decays over time

### The Gompertz growth function

As discussed above, the growth of a plant will slow down if its efficiency of synthesising new tissue declines (or inefficiency increases) even if the resources are non-limiting. For example, a very tall tree may slow down its height growth because of the physical constraints (i.e. reduced transport efficiency) it is facing in getting the resources (e.g., water) to where they are needed - the meristems, even if it is physiologically young and active and the resources like water and nitrogen are abundant in the soil. This transport inefficiency increases over time as the tree gets taller. As a result of this constraint, the growth efficiency (RGR) declines correspondingly over time.  This case can be expressed as:

- Eqn 5.1.

\begin{align}
  \frac{dW}{dt} &= r W \\
  \frac{dr}{dt} &= -\alpha r
\end{align}

where $W$ represents biomass, $r$ is the relative growth rate (RGR), and $\alpha$ is the decay rate of growth efficiency over time. This set of differential equations can be integrated into what is known as the Gompertz equation. The Gompertz growth model is a popular choice for modeling tumor growth in oncology by assuming that the growth rate declines as the cell mass multiplies

- Eqn 5.2.

$$
W = W_0 \exp \left[ \frac{r_0}{\alpha} (1 - e^{-\alpha t}) \right]
$$

This is the integrated form of Gompertz equation where $r_0$ is the intrinsic rate of growth without any decline in efficiency and $W_0$ represents initial biomass (g).

- Table 5.1.

| Symbol | Value | Units | Description |
| :- | :- | :- | :- |
| $t$ | - | $\mathrm{d}$ | Time unit used in the model |
| $r_0$ | 0.24 | $\mathrm{g}$ $\mathrm{g^{-1}}$ $\mathrm{d^{-1}}$ | Maximum $r$, intrinsic growth rate |
| $W_0$ | 0.15 | $\mathrm{g}$ | Seed mass or initial biomass |
| $\alpha$ | 0.03 | $\mathrm{d^{-1}}$ | Decay rate of the growth efficiency over time |
| $r$ | - | $\mathrm{g}$ $\mathrm{g^{-1}}$ $\mathrm{d^{-1}}$ | Relative growth rate |
| $W$ | - | $\mathrm{g}$ | Whole-plant biomass as dry matter |

#### Ex 5.1.1.

Create a system that implements the differential form of the Gompertz equation.

In [ ]:
@system G_diff(Controller) begin
    t(context.clock.time) ~ track(u"d")

#     r0: intrinsic_growth_rate => 0.24 ~ preserve(parameter, u"g/g/d")
#     W0: initial_biomass       => 0.15 ~ preserve(parameter, u"g")
#     α:  decay_rate            => 0.03 ~ preserve(parameter, u"d^-1")

#     r(α, r): actual_growth_rate => -α*r ~ accumulate(init = r0, u"g/g/d")
#     W(r, W): biomass            => r*W  ~ accumulate(init = W0, u"g")
    
    #= .. =#: intrinsic_growth_rate => #= .. =# ~ preserve(parameter, #= .. =#)
    #= .. =#: initial_biomass       => #= .. =# ~ preserve(parameter, #= .. =#)
    #= .. =#: decay_rate            => #= .. =# ~ preserve(parameter, #= .. =#)

    r(#= .. =#): actual_growth_rate => #= .. =# ~ accumulate(init = #= .. =#, #= .. =#)
    W(#= .. =#): biomass            => #= .. =# ~ accumulate(init = #= .. =#, #= .. =#)    
end

#### Ex 5.1.2.

Create a system that implements the integrated form (i.e. analytical solution) of the Gompertz equation.

In [ ]:
@system G_inte(Controller) begin
    t(#= .. =#) ~ track((#= .. =#)

    #= .. =#: intrinsic_growth_rate => #= .. =# ~ #= .. =#
    #= .. =#: initial_biomass       => #= .. =# ~ #= .. =#
    #= .. =#:  decay_rate           => #= .. =# ~ #= .. =#

    W(#= .. =#): biomass => begin
        #= .. =#
    end ~ track(#= .. =#)

end

#### Ex 5.1.3.

Simulate both forms of the Gompertz model for 300 days using daily steps. Plot and compare the results.

* Before running simulations, predict if you expect the results from both models to be identifical? Why or why not?
* Do your simulations support your predictions? Why or why not?

In [ ]:
visualize([G_inte, G_diff], :t, :W; kind = :line, names = ["W_analytical", "W_diff (1 day)"], config = c0, stop = 300u"d", xlim=(0,300))

#### Ex 5.1.4.

Modify your models to run on hourly steps, and compare the results. 

* Do you notice any differences between daily and hourly step runs? Explain why?
* How would overcome any pitfalls you've identified in this exercise?

In [ ]:
c0 = :Clock => :step => 1u"d"
stop = 300u"d"
y0 = simulate(G_diff; stop, config = c0)
y1 = simulate(G_inte; stop, config = c0)
y2 = simulate(G_diff; stop, config = :Clock => :step => 0.5u"d")
y3 = simulate(G_diff; stop, config = :Clock => :step => 1u"hr");

In [ ]:
p = visualize(y1, :t, :W; kind = :line, name = "W_analytical solution")
visualize!(p, y0, :t, :W; kind = :line, name = "W_diff (1 d)")
visualize!(p, y2, :t, :W; kind = :line, name = "W_diff (0.5 d)")
visualize!(p, y3, :t, :W; kind = :line, name = "W_diff (1 hr)")

### Errors due to numerical integration
Why are there differences between the analytical solution and numerical solutions with different integration steps? Do you expect similar differences for the logistic model (We briefly looked at this in L02)? Why or wy not?

In [ ]:
@system L_diff(Controller) begin(
    t(context.clock.time) ~ track(u"d")
    
    r: relative_growth_rate => 0.05 ~ preserve(u"g/g/d", parameter)
    W0: initial_biomass => 0.25 ~ preserve(u"g", parameter)
    Wf: potential_final_biomass => 300 ~ preserve(u"g", parameter)
    
    W(r, W, Wf): biomass => begin
        #= .. =#
    end ~ accumulate(u"g", init=W0)
        
end

In [ ]:
@system L_inte(Controller) begin
    t(context.clock.time) ~ track(u"d")
    
    r: relative_growth_rate => 0.05  ~ preserve(u"g/g/d", parameter)
    W0: initial_biomass => 0.25 ~ preserve(u"g", parameter)
    Wf: potential_final_biomass => 300 ~ preserve(u"g", parameter)
    
    W(r, W0, Wf, t): biomass => begin
        Wf / (1 + (Wf/W0 - 1)* exp(-r*t))
    end ~ track(u"g")
end

In [ ]:
z0 = simulate(L_diff; stop, config = c0)
z1 = simulate(L_inte; stop, config = c0)
z2 = simulate(L_diff; stop, config = :Clock => :step => 1u"hr")
z3 = simulate(L_diff; stop, config = :Clock => :step => 10u"d");

In [ ]:
q = visualize(z0, :t, :W; kind = :line, name = "W_analytical")
visualize!(q, z1, :t, :W; kind = :line, name = "W_diff (1 d)")
visualize!(q, z2, :t, :W; kind = :line, name = "W_diff (1 hr)")
visualize!(q, z3, :t, :W; kind = :line, name = "W_diff (10 d)")

## 5.3. Physiological processes governing plant growth

### Photosynthesis and respiration

Let's think about ways to talyor a simple growth function in the form of $\frac{dW}{dt}=rW$ to account for the basic physiological mechanisms that are specific to plants. Three key physiological processes that determine the rate of plant growth are photosynthesis, respiration, and allocation of assimilates. Photosynthesis assimilates CO<sub>2</sub> to provide carbohydrates as the substrate for respiration. Respiration breaks down the photosynthates to supply energy needed for maintenance of existing biomass as well as C skeletons and energy needed for construction of new biomass (growth). In its simplest form to incorporate the relationship between photosynthesis and respiration, the rate of biomass accumulation can be written as:

- Eqn 5.3.

$$
\frac{dW}{dt} = rW = c (P_g - R_{plant})
$$
  
See Table 5.1 in the handout for description of each term. 

Thornley (1970) illustrated elegantly how the above relationship translates to RGR, photosynthesis, and two components of respiration (growth and maintenance) as follows. This is a simple but robust representation of the photosynthesis-respiration relationship commonly applicable to all plant taxa.

- Eqn 5.4.

$$
r = Y_g \cdot c_1 \frac{P_g - k_m W}{W} = Y_g \cdot c_1 \left( \frac{P_g}{W} - k_m \right) = Y_g \cdot c_1 \left( P_{g_m} - k_m \right)
$$
  
As shown in section 5.1 with the Gompertz equation, the growth yield ($Y_g$) may decline over time as the plant ages.

- Table 5.1.

| Symbol | Value | Units | Description |
| :- | :- | :- | :- |
| $t$ | - | $\mathrm{d}$ | Time unit used in the model |
| $\alpha$ | 0.03 | $\mathrm{d^{-1}}$ | Decay rate of the growth efficiency over time |
| $Y_{g_0}$ | 0.75 | - | Potential growth yield of the biomass conversion process |
| $Y_g$ | - | - | Actual growth yield of the biomass conversion process |
| $c_1$ | 0.889 | - | Conversion factor from $\mathrm{CH_2O}$ to biomass assuming 45% carbon in biomass |
| $P_g$ | - | $\mathrm{g}$ $\mathrm{d^{-1}}$ | Substrate (as $\mathrm{C}$, $\mathrm{CO_2}$, $\mathrm{CH_2O}$) production rate by a whole-plant per unit time |
| $P_{g_m}$ | 0.4 | $\mathrm{g}$ $\mathrm{g^{-1}}$ $\mathrm{d^{-1}}$ | Max substrate (as $\mathrm{CH_2O}$) production rate per whole-plant mass per unit time |
| $k_m$ | 0.015 | $\mathrm{g}$ $\mathrm{g^{-1}}$ $\mathrm{d^{-1}}$ | Coefficient of maintenance respiration |
| $r$ | - | $\mathrm{g}$ $\mathrm{g^{-1}}$ $\mathrm{d^{-1}}$ | Relative growth rate |
| $W_0$ | 0.15 | $\mathrm{g}$ | Seed mass or initial biomass |
| $W$ | - | $\mathrm{g}$ | Whole-plant biomass as dry matter |

#### Ex 5.2.

Implement the growth model by Thornley (1970) into a system in Cropbox and examine its behavior using the information provided in the Table 5.1. Assume the $Y_g$ declines over time at a rate that given by $\alpha$. Mull over if this assumption is valid and if so, what plant or canopy process can make this decline over time.

* Evaluate the dependency of your model using `dependency()` function of the Cropbhox framework.
* List the parameters and their values using `parameters()` function of the Cropbox framework.
* Run your simulation on daily step for 300 days.
* Compare the results with those from Ex 5.1. 

In [ ]:
@system Thornley1970(Controller) begin
    t(context.clock.time) ~ track(u"d")

    α:         decay_rate             => 0.03  ~ preserve(parameter, u"d^-1")
    Yg0:       potential_growth_yield => 0.75  ~ preserve(parameter)
    Yg(α, Yg): growth_efficiency      => -α*Yg ~ accumulate(init = Yg0)

    c1:   conversion_factor        => 12/30/0.45 ~ preserve(parameter)
    
    β:         production_decay           => 0.0  ~ preserve(parameter, u"d^-1")
    Pg_m0: max_production_rate_per_mass => 0.4        ~ preserve(parameter, u"g/g/d")
    Pg_m(β, Pg_m): production_rate_per_mass => -β*Pg_m ~ accumulate(init=Pg_m0, u"g/g/d")
    km:   maintenance_coeff        => 0.015      ~ preserve(parameter, u"g/g/d")

    r(#= .. =#): intrinsic_RGR => begin
        #= .. =#
    end ~ track(#= .. =#)

    W0:      initial_biomass => #= .. =# ~ preserve(parameter, #= .. =#)
    W(#= .. =#): biomass     => #= .. =# ~ accumulate(init = #= .. =#, #= .. =#)

end

In [ ]:
Cropbox.dependency(Thornley1970)

In [ ]:
parameters(Thornley1970)

In [ ]:
c = :Clock => :step => 1u"d"
t70 = simulate(Thornley1970, config = c, stop = 300u"d")

In [ ]:
visualize(t70, :t, [:W], kind = :line)

### Carbon allocation model

Allocation or partitioning of the photosynthates but not represented in the above model is a critical element to determine their destination and fate as to how and in what part they will be used. For example, some assimilates will be transported to roots to function as subsrates for providing ATPs to maintain the ion gradients while others can be invested back to build more new leaves. More investment in photosynthetic machinery (i.e. allocation to foliage) will yield greater return of biomass at a faster rate; this is similar to the growth of bank account balance with the compound interest rate. 

Monsi (1960)'s model as summarized in Kim et al. (2020) intuitively demonstrates the importance of photosynthate allocation in plant growth.

- Eqn 5.5.

$$
r = c_2 (m A - n R)
$$
  
Both models are simple but robustly grounded in physiology to provide critical insights for understanding the intricate relationship between these three processes and their trade-offs determining plant growth. The trade-offs of allocating photosynthates between photosynthetic and non-photosynthetic parts can be particularly insightful and relevant for the growth of crops like lettuce and cabbage for which the foliage biomass accounts for the majority of harvested yield or crops with large underground storage organs like radish, potato, garlic, and onions. Carbon allocation to non-photosynthetic parts is also functionally important in  herbaceous perennials with storage organs such as rhizomes and woody plants for which dead parts (i.e. wood and bark) continue to be part of the whole-plant biomass.

- Table 5.1.

| Symbol | Value | Units | Description |
| :- | :- | :- | :- |
| $t$ | - | $\mathrm{d}$ | Time unit used in the model |
| $c_2$ | 0.606 | - | Conversion factor from $\mathrm{CO_2}$ to biomass assuming 45% carbon in biomass |
| $m$ | 0.5 | - | Fraction of photosynthetic biomass |
| $n$ | 0.5 | - | Fraction of non-photosynthetic biomass ($1 - m$) |
| $A_{\mathrm{max}}$ | 0.08 | $\mathrm{g}$ $\mathrm{g^{-1}}$ $\mathrm{hr^{-1}}$ | Maximal net $\mathrm{CO_2}$ assimilation rate |
| $R_{\mathrm{max}}$ | 0.006 | $\mathrm{g}$ $\mathrm{g^{-1}}$ $\mathrm{hr^{-1}}$ | Maximal respiration rate |
| $\alpha$ | 0.03 | $\mathrm{d^{-1}}$ | Decay rate of the growth efficiency over time |
| $A$ | - | $\mathrm{g}$ $\mathrm{g^{-1}}$ $\mathrm{d^{-1}}$ | Actual net $\mathrm{CO_2}$ assimilation rate per day during photoperiod |
| $R$ | - | $\mathrm{g}$ $\mathrm{g^{-1}}$ $\mathrm{d^{-1}}$ | Actual respiration rate per day |
| $D_L$ | 12 | $\mathrm{hr}$ | Day length (i.e. photoperiod) |
| $D$ | 24 | $\mathrm{hr}$ | Total hours per day |
| $r$ | - | $\mathrm{g}$ $\mathrm{g^{-1}}$ $\mathrm{d^{-1}}$ | Relative growth rate |
| $W_0$ | 0.15 | $\mathrm{g}$ | Seed mass or initial biomass |
| $W$ | - | $\mathrm{g}$ | Whole-plant biomass as dry matter |

#### Ex 5.3.

Implement the growth model by Monsi (1960) into a system in Cropbox and examine its behavior using the information provided in the Table 5.1. Assume both $A$ and $R$ declines over time at a rate given by $\alpha$. Also note that 1) $A$ and $R$ are expressed in $g$ of CO<sub>2</sub> in hourly unit and 2) photosynthesis requires light.  

* Evaluate the dependency of your model using `dependency()` function of the Cropbhox framework.
* List the parameters and their values using `parameters()` function of the Cropbox framework.
* Run your simulation on daily step for 300 days.
* Compare the results with those from Ex 5.1. 

In [ ]:
@system Monsi1960(Controller) begin
    t(context.clock.time) ~ track(u"d")

    c2: conversion_factor => 12/44/0.45 ~ preserve(parameter)    

    m:    leaf_mass_ratio   => #= .. =# ~ preserve(parameter)
    n(m): non_leaf_fraction => #= .. =# ~ track

    Amax: maximal_assimilation => 0.08  ~ preserve(parameter, u"g/g/hr")
    Rmax: maximal_respiration  => 0.005 ~ preserve(parameter, u"g/g/hr")
    α:    decay_rate           => 0.03  ~ preserve(parameter, u"d^-1")

    A(α, A): hourly_assimilation_per_mass => -α*A ~ accumulate(init = Amax, u"g/g/hr")
    R(α, R): hourly_respiration_per_mass  => -α*R ~ accumulate(init = Rmax, u"g/g/hr")

    DL: day_length  => 12 ~ preserve(parameter, u"hr")
    D:  total_hours => 24 ~ preserve(parameter, u"hr")

    A′(A, DL, D): assimilation_per_mass => A * (#= .. =#) ~ track(u"g/g/d")
    R′(R):        respiration_per_mass                ~ track(u"g/g/d")
    
    r(#= .. =#): RGR => #= .. =# ~ track(#= .. =#)

    W0:      initial_biomass => 0.15 ~ preserve(parameter, u"g")
    W(#= .. =#): biomass     => #= .. =# ~ accumulate(init = #= .. =#, #= .. =#)
end

In [ ]:
parameters(Monsi1960)

In [ ]:
Cropbox.dependency(Monsi1960)

In [ ]:
m60 = simulate(Monsi1960, config = c, stop = 300u"d");

In [ ]:
visualize(m60, :t, :W, kind=:line)

In [ ]:
p = visualize(t70, :t, :W, kind = :line, name = "Thornley (1970)")
visualize!(p, m60, :t, :W, kind = :line, name = "Monsi (1960)")

### Senescence and decline of biomass

Plants, especially annuals, will lose their biomass. Cells, tissues, and organs will senesce, die, and disintegrate eventually, and are no longer part of the living "biomass". The models we've looked at so far do not account for this aspect of senescence and death. Think about how the processes and mechanisms associated with senescence and death of plant can be implemented in these models. Some useful information may be found in Table 1.

- Table 5.1.

| Symbol | Value | Units | Description |
| :- | :- | :- | :- |
| $\alpha_{\mathrm{sen}}$ | 100 | $\mathrm{d}$ | Plant age (days) at which senescence begins |
| $k_s$ | 0.01 | $\mathrm{g}$ $\mathrm{g^{-1}}$ $\mathrm{d^{-1}}$ | Senescence rate of degradable biomass over time |
| $f_d$ | 1.0 | - | Fraction of degradable biomass |
| ... | - | - | ... |

Here's an example of how senescence process may be incorporated into the Monsi model.

In [ ]:
@system MyPlantModel(Controller) begin
    t(context.clock.time) ~ track(u"d")

    c2: conversion_factor => 12/44/0.45 ~ preserve(parameter)    

    m:    leaf_mass_ratio   => 0.5 ~ preserve(parameter)
    n(m): non_leaf_fraction => 1-m ~ track

    Amax: maximal_assimilation => 0.08 ~ preserve(parameter, u"g/g/hr")
    Rmax: maximal_respiration  => 0.006 ~ preserve(parameter, u"g/g/hr")

#    α:       decay_rate                   => 0.03 ~ preserve(parameter, u"d^-1")
    α_max: maximum_decay_rate => 0.03    ~ preserve(parameter, u"d^-1")
    k: delay_coeff            => 0.01    ~ preserve(parameter, u"d")    
    α0: initial_decay         => 0.0     ~ preserve(parameter, u"d^-1")
    
    α(k,α_max,α): decay_rate    => k*(α_max - α) ~ accumulate(init = α0 , u"d^-1")

    A(α, A): hourly_assimilation_per_mass => -α*A ~ accumulate(init = Amax, u"g/g/hr")
    R(α, R): hourly_respiration_per_mass  => -α*R ~ accumulate(init = Rmax, u"g/g/hr")

    DL: day_length  => 12 ~ preserve(parameter, u"hr")
    D:  total_hours => 24 ~ preserve(parameter, u"hr")

    A′(A, DL, D): assimilation_per_mass => A * (DL/D) ~ track(u"g/g/d")
    R′(R):        respiration_per_mass                ~ track(u"g/g/d")

    r(c2, m, n, A′, R′): RGR => c2 * (m*A′ - n*R′) ~ track(u"g/g/d")

    s_max: maximum_senescence_rate => 0.02    ~ preserve(parameter, u"d^-1")
    ks: senescence_delay_coeff     => 0.01    ~ preserve(parameter, u"d")    
    s0: initial_senescen           => 0.0     ~ preserve(parameter, u"d^-1")
    
    s(ks,s_max,s): senescence_rate    => ks*(s_max - s) ~ accumulate(init = s0 , u"d^-1")

    W0:          initial_biomass => 0.15       ~ preserve(parameter, u"g")
    W(r, s, W): biomass         => (r - s)*W ~ accumulate(init = W0, u"g")
end

## Homework Problems ##

### Homework 5.1
You will expand the models you worked with so far to build a model to satisfy the following conditions: 

Assuming that you are growing plants in a controlled growth chamber with the environmental conditions described in the table, build a simple plant growth model for a plant of your choice for which the entire above-ground biomass will be harvested at the end of one growing season like in a bioenergy crop; that is, no specific organ represents its yield but the entire shoot biomass does. Use the parameters and their values listed in Table~\ref{table:1} as appropriate for your model. Answer the following questions using your model or based on your model.

1. Decide a plant species for which you will develop the growth model. Why did you choose this species?
2. Implement model using Cropbox and plot the biomass trajectory over time using the parameter values provided in Table~\ref{table:1} as the initial parameter values. When is the maximum biomass reached and what is that biomass? Do the values look reasonable for the species of your choice based on your knowledge about the plant? 
3. Imagine that you are a plant breeder trying to increase the biomass that can be harvested at the end of the season for this energy crop. What breeding strategies (e.g., increase photosynthesis, increase leaf area) would you use? Can you test your ideas using the model you just developed? How? If not, why not?
4. Using your model, let's try all means available to increase the biomass at harvest. How much more biomass did you gain compared to the initial model prediction? How did you achieve that? Do you think you could have achieved the same outcome in other ways? If so, how or why not? 
5. Describe whether or not and how your model represents the properties and behavior of the system you are trying to simulate satisfactorily. What is your model good for and what are its shortcomings and limitations? 
